In [ ]:
# ==== 0. 준비 ====
import os
from dotenv import load_dotenv
load_dotenv()  # .env에서 OPENAI_API_KEY 로딩

# ==== 1. LangChain 기본 ====
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")  # 또는 최신 gpt-4.1 / gpt-4o / gpt-4o-mini 등
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{question}")
])

# LCEL: 프롬프트 → LLM → 출력
chain = prompt | llm
resp = chain.invoke({"question": "LangChain이 뭐예요? 한 문단으로 설명해줘."})
print(resp.content)

In [ ]:
# ==== 2. 미니 RAG(FAISS) ====
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 예시 문서
docs = [
    {"id": 1, "text": "LangChain enables building LLM apps with components like prompts, chains, and agents."},
    {"id": 2, "text": "FAISS provides efficient vector similarity search for embeddings."},
    {"id": 3, "text": "RAG combines retrieval with generation to ground model answers in your data."},
]
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = []
for d in docs:
    for c in splitter.split_text(d["text"]):
        texts.append(c)

emb = OpenAIEmbeddings()
vdb = FAISS.from_texts(texts, embedding=emb)

def retrieve(query, k=3):
    return vdb.similarity_search(query, k=k)

# RAG 체인: 질의 → 검색 → 컨텍스트+질의 → 답변
rag_prompt = ChatPromptTemplate.from_template(
    "Answer based on CONTEXT.\nCONTEXT:\n{context}\n\nQuestion: {question}"
)
rag_chain = (
    {"context": lambda x: "\n".join([d.page_content for d in retrieve(x["question"])]),
     "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)
print(rag_chain.invoke({"question": "What is RAG and why use FAISS?"}))
